In [7]:
import os
import numpy as np
from tensorflow.keras.models import load_model

def let_record_audio():
    print("Recording audio for demonstration purposes...")
    # 여기에서 실제 녹음 코드는 실행되지 않지만, 녹음 중인 것처럼 보입니다.
    print("recording is finished")

def load_saved_model(model_path):
    model_best = load_model(model_path)
    model_best.summary()
    return model_best

def preprocess_files(file_list_pitch, path):
    reshaped_files = []
    
    for file in file_list_pitch:
        file_path = os.path.join(path, file)
        if os.path.exists(file_path):
            data = np.load(file_path)
            reshaped_data = data.reshape(1, -1, 1) # Data reshaping, flexible to the time dimension
            reshaped_files.append(reshaped_data)
        else:
            print(f"File {file_path} not found.")
    
    if reshaped_files:
        return np.vstack(reshaped_files).astype('float32')
    else:
        print("No data to stack.")
        return np.array([])

def predict_with_model(model_best, file_list_pitch, path):
    data = preprocess_files(file_list_pitch, path)
    
    if data.size > 0:
        probabilities = model_best.predict(data)
        labels = [1 if prob[0] >= 0.5 else 0 for prob in probabilities]
        print("Probabilities and Labels for Each File:")
        
        # 각 파일마다 예측 확률과 레이블을 출력합니다.
        for file_name, prob, label in zip(file_list_pitch, probabilities, labels):
            print(f"File {file_name}: Probability for class 0: {1 - prob[0]:.4f}, "
                  f"Probability for class 1: {prob[0]:.4f}, Label: {label}")
            
        return labels, probabilities
    else:
        print("No valid data to predict.")
        return None, None

def evaluate_voice_phishing_risk(probabilities, labels):
    num_voice_phishing_files = 0
    sum_class_0_prob_voice_phishing = 0.0

    for idx, label in enumerate(labels):
        if label == 0 and 1 - probabilities[idx][0] >= 0.5:  # Lower the threshold to 0.5
            num_voice_phishing_files += 1
            sum_class_0_prob_voice_phishing += 1 - probabilities[idx][0]

    total_num_files = len(labels)
    if num_voice_phishing_files > 0:
        probability_P = (num_voice_phishing_files / total_num_files) * (sum_class_0_prob_voice_phishing / num_voice_phishing_files)
    else:
        probability_P = 0.0

    if 0.2 <= probability_P < 0.4:
        risk_level = "보이스피싱 경고"
    elif 0.4 <= probability_P < 0.6:
        risk_level = "보이스피싱 위험"
    elif 0.6 <= probability_P <= 1.0:
        risk_level = "보이스피싱 매우 위험"
    else:
        risk_level = "정상"

    print("Number of voice phishing files:", num_voice_phishing_files)
    print("Total number of files:", total_num_files)
    print("Probability P:", probability_P)
    print("Risk Level:", risk_level)

def main():
    let_record_audio()
    
    model_best = load_saved_model('D:\\user\\Downloads\\230809_pitch_model (1).h5')
    
    # 첫 번째 폴더 경로
    path1 = 'D:\\user\\Downloads\\pitch_counsel_시연_npy'
    
    print("\n=== Counsel pitch data path ===\n")
    process_and_evaluate(model_best, path1)
    
    # 두 번째 폴더 경로
    path2 = 'D:\\user\\Downloads\\pitch_vp_시연_npy'
    
    print("\n=== Voice phishing pitch data path ===\n")
    process_and_evaluate(model_best, path2)

def process_and_evaluate(model_best, path):
    file_list = os.listdir(path)
    file_list_pitch = [file for file in file_list if file.endswith('.npy')]
    
    labels, probabilities = predict_with_model(model_best, file_list_pitch, path)
    
    if labels is not None and probabilities is not None:
        evaluate_voice_phishing_risk(probabilities, labels)

if __name__ == '__main__':
    main()

Recording audio for demonstration purposes...
recording is finished
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 157, 1)]          0         
                                                                 
 normalization (Normalizatio  (None, 157, 1)           315       
 n)                                                              
                                                                 
 Conv1D_1 (Conv1D)           (None, 157, 32)           128       
                                                                 
 Conv1D_2 (Conv1D)           (None, 157, 64)           6208      
                                                                 
 Conv1D_3 (Conv1D)           (None, 157, 128)          24704     
                                                                 
 Conv1D_4 (Conv1D)           (None, 157, 128)          4928